# Geração de Relatórios do Harmony Endpoint

## Preparação do Ambiente

### Importação de Bibliotecas

In [29]:
from chkp_harmony_endpoint_management_sdk import HarmonyEndpoint, InfinityPortalAuth, HarmonyEndpointSDKInfo
import os
import pandas as pd

### Checagem do Software Deloper Kit (SDK)

In [30]:
sdk_info: HarmonyEndpointSDKInfo = HarmonyEndpoint.info()
sdk_info = str(sdk_info).split(',')
for info in sdk_info:
    print(info.strip())

sdk_build:"9435737705"
sdk_version:"1.1.38"
spec:"web-mgmt-external-api-production"
spec_version:"1.9.212"
released_on:"2024-06-09T10:07:06.796444"


### Inicializando a conexão com o Harmony Endpoint

In [31]:
def load_configuration():
    client_id = os.getenv('CLIENT_ID')
    api_key = os.getenv('API_SECRET')
    api_url = os.getenv('API_URL')
    return client_id, api_key, api_url

In [32]:
he = HarmonyEndpoint()
client_id, api_key, api_url = load_configuration()

This package is currently in EA (Early Availability) stage. Use with caution, as it may undergo significant changes and improvements. Feedback and contributions are highly encouraged. To report a bug, please go to https://github.com/CheckPointSW/harmony-endpoint-management-py-sdk/issues or contact us at harmony-endpoint-external-api@checkpoint.com


In [33]:
he.connect(infinity_portal_auth=InfinityPortalAuth(
        client_id=client_id, # O "Client ID"
        access_key=api_key, # A "Secret Key"
        gateway=api_url # A "Authentication URL"
        )) 

## Geração de Metadados

### Coleta e Estruturação de Metadados de Todas as Políticas

In [34]:
print("Requisitando metadados de todas as políticas")
rules_metadata_res = he.policy_general_api.get_all_rules_metadata(header_params={ "x-mgmt-run-as-job": 'on'})
rules_metadata = rules_metadata_res.payload

rules = []
for item in rules_metadata:
    record = {
        "Orientation": item["orientation"],
        "ID": item["id"],
        "Name": item["name"],
        "Connection State": item["connectionState"],
        "Family": item["family"],
        "Is Default Rule": item["isDefaultRule"],
        "Order": item["order"]
    }
    rules.append(record)

df_rules = pd.DataFrame(rules)
df_rules

Requisitando metadados de todas as políticas


,Orientation,ID,Name,Connection State,Family,Is Default Rule,Order
0,DEVICE,00000000-0000-0000-0000-000000fffaaa@00000000-...,Default settings for the entire organization,CONNECTED,Access,True,0
1,DEVICE,4e40c146-2a79-4872-a66f-ae3cc1a7999d@5a402819-...,ACT-LAB-Servers,CONNECTED,Threat Prevention,False,0
2,DEVICE,63a73f48-7448-477d-9dc9-ecb88650b55d@9f5774a7-...,ACT-LAB-Desktops,CONNECTED,Threat Prevention,False,1
3,DEVICE,90cdde52-bdc1-4c99-aff8-7568a43ecbb2@ccdf092e-...,Windows desktop,CONNECTED,Threat Prevention,False,2
4,DEVICE,6471c8c5-cc40-47be-8e6a-8c7e38b67ce8@b37040e2-...,Windows Server,CONNECTED,Threat Prevention,False,3
5,DEVICE,5e222ece-a39e-466f-b5d0-820de41b14db@bf591836-...,FDE-WIN-Desktop,CONNECTED,Threat Prevention,False,4
6,DEVICE,00000000-0000-0000-0000-000000101111@1c7d3e9d-...,Default rule,CONNECTED,Threat Prevention,True,5
7,DEVICE,5df4d66c-4789-4617-af4f-60a46d25d7e8@81b6db71-...,FDE-ME-PP,CONNECTED,Data Protection,False,0
8,DEVICE,00000000-0000-0000-0000-000000fde030@e427e7bf-...,Default settings for the entire organization,CONNECTED,Data Protection,True,1
9,USER,00000000-0000-0000-0000-000010fde030,Default settings for the entire organization,CONNECTED,OneCheck,True,0


### Filtragem dos Identificadores das Policies de Threat Preventions

In [35]:
threat_prevention_rules_id = []

for item in rules_metadata:
    if item["family"] == 'Threat Prevention':
        threat_prevention_rules_id.append(item["id"])
        
threat_prevention_rules_id

['4e40c146-2a79-4872-a66f-ae3cc1a7999d@5a402819-10a1-47b2-9f24-1b2f39c14cfe@8b56ff70-5b0e-48cb-96f5-905e508ba2d3@d7166c50-8195-4999-89c3-c95004b0e0ec',
 '63a73f48-7448-477d-9dc9-ecb88650b55d@9f5774a7-edfa-4556-9ed9-1994173fa3d5@a47ff5aa-2568-4d4a-9594-8e4885769e52@e4512197-5de9-44f3-9628-5ce9bb923e48',
 '90cdde52-bdc1-4c99-aff8-7568a43ecbb2@ccdf092e-43bd-45b9-8ebf-1517ab448318@dd699df3-6998-4b17-8d0a-0816fa333354@f02bab05-6bf5-43ad-aa50-3e993a738ca6',
 '6471c8c5-cc40-47be-8e6a-8c7e38b67ce8@b37040e2-20e1-4486-9fe1-29b119445b12@bc762a46-6c34-46ba-bad2-fdadaa8315ee@cf6c09d0-956f-4f8c-b404-3ddb8b0f1344',
 '5e222ece-a39e-466f-b5d0-820de41b14db@bf591836-62b3-423f-9f36-fa160cc1f3c2@ebca1cb7-b6bb-4e40-959b-3d811bb0ccee@f0a5612b-b165-4504-af40-964b4223c4c5',
 '00000000-0000-0000-0000-000000101111@1c7d3e9d-3aad-4ef8-870a-f01efa586628@c93b7053-5378-4415-8d62-1a4efaca68e8@ef1134f4-06c1-4922-b217-37b8c24427e5']

### Coleta e Estruturação de Dados das Políticas de Threat Prevention

In [36]:
threat_prevention_rules_statuses = []

for id in threat_prevention_rules_id:
    threat_prevention_policy_status_res = he.policy_threat_prevention_api.get_threat_prevention_policy(
        header_params={ "x-mgmt-run-as-job": 'on'},
        query_params={"detailsLevel": "NORMAL"},
        path_params={"ruleId": id}
    )
    threat_prevention_policy_status = threat_prevention_policy_status_res.payload
    print(threat_prevention_policy_status)
    
    network = threat_prevention_policy_status['policy']['protections']['network']
    system = threat_prevention_policy_status['policy']['protections']['system']
    behavioral = system['behavioral']['settings']
    record = {
        "Policy ID": id,
        "Files Threat Emulation Mode": network['files']["settings"]["emulation"]["mode"],
        "Download Emulation Extraction": network['files']["settings"]["emulation"]["mode"],
        "Password Reuse": network["passwordReuse"]["settings"]["protectionMode"],
        "Zero-Phishing": network["phishing"]["settings"]["protectionMode"],
        "URL Filtering": network["urlFiltering"]["settings"]["mode"],
        "Anti-Malware": system["malware"]["settings"]["antiMalwareMode"],
        "Anti-Exploit": system["exploit"]["settings"]["mode"],
        "Anti-Ransomware": behavioral["antiRansomware"]["mode"],
    }
    threat_prevention_rules_statuses.append(record)

df_rules_threat_prevention = pd.DataFrame(threat_prevention_rules_statuses)
df_rules_threat_prevention

{'policy': {'protections': {'network': {'files': {'metadata': {'blade': 'THREAT_EMULATION'}, 'settings': {'browser': {'extension': {'customSettings': [], 'pinToToolbar': True, 'anonymousBrowsingMode': 'DEFER', 'chromeExtensionId': 'deakbjemijlmlcehdgejmdpekkceodmk', 'enableBrowserExtensions': True}}, 'emulation': {'mode': 'PREVENT', 'download': {'fileActions': {'supported': 'EXTRACT_SUSPEND', 'unsupported': 'ALLOW_DOWNLOAD', 'emulatedOnly': 'EMULATE_SUSPEND'}, 'notificationsMode': 'ENABLED', 'threatExtraction': {'mode': 'EXTRACT_ELEMENTS'}}, 'upload': {'mode': 'PREVENT', 'fileActions': {'supported': 'EMULATE_BACKGROUND', 'unsupported': 'ALLOW_UPLOAD', 'fileTypeOverrides': []}, 'notificationsMode': 'ENABLED', 'fileProtectionDomainsActions': [], 'maxFileSize': 15, 'protectedDomains': [], 'protectedIPs': []}, 'emulationEnvironment': {'type': 'CLOUD', 'applianceSettings': {'fallbackToCloud': True, 'appliances': []}, 'environmentMode': 'RECOMMEND', 'images': []}, 'sensors': {'enabled': True

,Policy ID,Files Threat Emulation Mode,Download Emulation Extraction,Password Reuse,Zero-Phishing,URL Filtering,Anti-Malware,Anti-Exploit,Anti-Ransomware
0,4e40c146-2a79-4872-a66f-ae3cc1a7999d@5a402819-...,PREVENT,PREVENT,PREVENT,PREVENT,PREVENT,PREVENT,PREVENT,PREVENT
1,63a73f48-7448-477d-9dc9-ecb88650b55d@9f5774a7-...,PREVENT,PREVENT,PREVENT,PREVENT,PREVENT,PREVENT,PREVENT,PREVENT
2,90cdde52-bdc1-4c99-aff8-7568a43ecbb2@ccdf092e-...,PREVENT,PREVENT,PREVENT,PREVENT,PREVENT,PREVENT,PREVENT,PREVENT
3,6471c8c5-cc40-47be-8e6a-8c7e38b67ce8@b37040e2-...,PREVENT,PREVENT,PREVENT,PREVENT,PREVENT,PREVENT,PREVENT,PREVENT
4,5e222ece-a39e-466f-b5d0-820de41b14db@bf591836-...,PREVENT,PREVENT,PREVENT,PREVENT,PREVENT,PREVENT,PREVENT,PREVENT
5,00000000-0000-0000-0000-000000101111@1c7d3e9d-...,PREVENT,PREVENT,PREVENT,PREVENT,DETECT,PREVENT,OFF,PREVENT


## Relatório de Configurações das Políticas de Threat Prevention

In [37]:
categories = [
    "URL Filtering", "Anti-Exploit", "Download Emulation Extraction", 
    "Password Reuse", "Files Threat Emulation Mode", "Anti-Ransomware", 
    "Zero-Phishing", "Anti-Malware"
]

counters = { category: {'DETECT': 0, 'PREVENT': 0, 'OFF': 0} for category in categories}

for rule_status in threat_prevention_rules_statuses:
    for category in categories:
        mode = rule_status.get(category, "OFF")
        counters[category][mode] += 1

df_report = pd.DataFrame(counters).T
df_report

,DETECT,PREVENT,OFF
URL Filtering,1,5,0
Anti-Exploit,0,5,1
Download Emulation Extraction,0,6,0
Password Reuse,0,6,0
Files Threat Emulation Mode,0,6,0
Anti-Ransomware,0,6,0
Zero-Phishing,0,6,0
Anti-Malware,0,6,0
